Analyze Dataset from overnight runs

In [5]:
import pandas as pd

data = pd.DataFrame([[60424, 4096],
[76524, 4096],
[37008, 2048],
[65964, 4096],
[36456, 2048],
[3924, 256],
[35092, 2048],
[123296, 8192],
[79392, 4096],
[71724, 4096],
[36424, 2048],
[36116, 2048],
[18344, 1024],
[77152, 4096],
[57040, 4096],
[15072, 1024],
[76420, 4096],
[113608, 8192],
[61344, 4096],
[79848, 4096],
[52092, 4096],
[24988, 2048],
[80320, 4096],
[26460, 2048],
[28348, 2048],
[24016, 1024],
[111924, 8192],
[63596, 4096],
[60956, 4096],
[32592, 2048],
[36212, 2048],
[57040, 4096],
[36524, 2048],
[32348, 2048],
[16132, 1024],
[67820, 4096],
[80444, 4096],
[15840, 1024],
[36460, 2048],
[78552, 4096],
[72040, 4096],
[37204, 2048],
[59496, 4096],
[70104, 4096],
[58016, 4096],
[32656, 2048],
[27536, 2048],
[81744, 4096],
[36044, 2048],
[34944, 2048],
[72424, 4096],
[34872, 2048],
[47728, 4096],
[34840, 2048],
[37152, 2048],
[60636, 4096],
[76804, 4096],
[36156, 2048],
[78056, 4096],
[111888, 8192],
[76784, 4096],
[34328, 2048],
[50228, 4096],
[36848, 2048],
[32364, 2048],
[31480, 2048]],
columns=['score', 'max_tile'])

num_over_1024 = len(data[data['max_tile']>=1024])
num_over_2048 = len(data[data['max_tile']>=2048])
num_over_4096 = len(data[data['max_tile']>=4096])
num_over_8196 = len(data[data['max_tile']>=8192])

print(f'Number of games over 1024: {num_over_1024/len(data)}')
print(f'Number of games over 2048: {num_over_2048/len(data)}')
print(f'Number of games over 4096: {num_over_4096/len(data)}')
print(f'Number of games over 8192: {num_over_8196/len(data)}')
print(len(data))

Number of games over 1024: 0.9848484848484849
Number of games over 2048: 0.9090909090909091
Number of games over 4096: 0.5151515151515151
Number of games over 8192: 0.06060606060606061
66


In [1]:
from Reporter import LightConcurrentReporter
from MarkovDPAI import *
import numpy as np

def perform_trial(parameters):
    path_penalty = parameters.get('path_penalty')
    loss_penalty = parameters.get('loss_penalty')
    score_factor = parameters.get('score_factor')

    ai_input=ExpectiMax7(depth = 5, path_pen=path_penalty, loss_penalty=loss_penalty, score_factor=score_factor)
    r = LightConcurrentReporter(ai_input, 50)
    report = pd.DataFrame(r.generate_report(), columns=['score','max_tile'])
    print(np.mean(report['score']), len(report[report['max_tile']>=2048])/50.0)
    return len(report[report['max_tile']>=2048])/50.0

In [2]:
import pandas as pd
from ax.service.ax_client import AxClient
from ax.modelbridge.registry import Models
from ax.modelbridge.generation_strategy import GenerationStrategy, GenerationStep

generation_strategy = GenerationStrategy(
    steps=[
        GenerationStep(model=Models.SOBOL, num_trials=10),
        GenerationStep(model=Models.GPEI, num_trials=-1),   # GPEI for the rest of the trials
    ]
)
ax_client = AxClient(generation_strategy=generation_strategy)
# ax_client = AxClient()
ax_client.create_experiment(name='optimize Params',
                            parameters=[
                                {'name': 'path_penalty',
                                    'type': 'range',
                                    'bounds': [0.0, 10.0]},
                                {'name': 'loss_penalty',
                                    'type': 'range',
                                    'bounds': [0.0, 10.0]},
                                {'name': 'score_factor',
                                    'type': 'range',
                                    'bounds': [0.0, 5.0]},
                            ],
)

for i in range(10):
    parameters, trial_index = ax_client.get_next_trial()
    ax_client.complete_trial(trial_index=trial_index, raw_data=perform_trial(parameters))


for i in range(100):
    parameters, trial_index = ax_client.get_next_trial()
    ax_client.complete_trial(trial_index=trial_index, raw_data=perform_trial(parameters))

best_parameters, metrics = ax_client.get_best_parameters()
print(f'The best parameters are: {best_parameters}')
print(f'with score {metrics}')

[WARNING 10-25 11:52:21] ax.service.utils.with_db_settings_base: Ax currently requires a sqlalchemy version below 2.0. This will be addressed in a future release. Disabling SQL storage in Ax for now, if you would like to use SQL storage please install Ax with mysql extras via `pip install ax-platform[mysql]`.
[INFO 10-25 11:52:21] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.
[INFO 10-25 11:52:21] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter path_penalty. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 10-25 11:52:21] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter loss_penalty. If that is not the expected value type, you can explicitly specify 'value_type' ('i

41105.04 0.84


[INFO 10-25 11:53:40] ax.service.ax_client: Completed trial 1 with data: {'objective': (0.84, None)}.
/home/kylemcmaster/.local/lib/python3.10/site-packages/ax/modelbridge/cross_validation.py:463: UserWarning: Encountered exception in computing model fit quality: RandomModelBridge does not support prediction.
  warn("Encountered exception in computing model fit quality: " + str(e))
[INFO 10-25 11:53:40] ax.service.ax_client: Generated new trial 2 with parameters {'path_penalty': 1.634202, 'loss_penalty': 3.538106, 'score_factor': 0.092499} using model Sobol.


36399.68 0.84


[INFO 10-25 11:53:51] ax.service.ax_client: Completed trial 2 with data: {'objective': (0.08, None)}.
/home/kylemcmaster/.local/lib/python3.10/site-packages/ax/modelbridge/cross_validation.py:463: UserWarning: Encountered exception in computing model fit quality: RandomModelBridge does not support prediction.
  warn("Encountered exception in computing model fit quality: " + str(e))
[INFO 10-25 11:53:51] ax.service.ax_client: Generated new trial 3 with parameters {'path_penalty': 8.181295, 'loss_penalty': 8.789798, 'score_factor': 4.264734} using model Sobol.


...

[INFO 10-25 13:09:07] ax.service.ax_client: Generated new trial 108 with parameters {'path_penalty': 9.215333, 'loss_penalty': 1.100342, 'score_factor': 5.0} using model GPEI.
[INFO 10-25 13:09:49] ax.service.ax_client: Completed trial 108 with data: {'objective': (0.9, None)}.


43096.24 0.9


[INFO 10-25 13:09:51] ax.service.ax_client: Generated new trial 109 with parameters {'path_penalty': 9.14172, 'loss_penalty': 7.018821, 'score_factor': 4.509579} using model GPEI.
[INFO 10-25 13:10:34] ax.service.ax_client: Completed trial 109 with data: {'objective': (0.92, None)}.


44787.84 0.92
The best parameters are: {'path_penalty': 4.538136591937287, 'loss_penalty': 10.0, 'score_factor': 1.8307266539837497}
with score ({'objective': 0.8791024169040762}, {'objective': {'objective': 0.00013430486593813665}})


/home/kylemcmaster/.local/lib/python3.10/site-packages/ax/utils/stats/model_fit_stats.py:199: RuntimeWarning: divide by zero encountered in divide
  return float(np.mean(1.96 * 2 * se_pred / np.abs(y_obs)))
/home/kylemcmaster/.local/lib/python3.10/site-packages/ax/utils/stats/model_fit_stats.py:229: RuntimeWarning: divide by zero encountered in scalar divide
  return float((np.max(y_obs) - min_y_obs) / min_y_obs)


In [15]:
for i in range(30):
    parameters, trial_index = ax_client.get_next_trial()
    ax_client.complete_trial(trial_index=trial_index, raw_data=perform_trial(parameters))
    
best_parameters, metrics = ax_client.get_best_parameters()
print(f'The best parameters are: {best_parameters}')
print(f'with score {metrics}')

/home/kylemcmaster/.local/lib/python3.10/site-packages/botorch/optim/optimize.py:568: RuntimeWarning: Optimization failed in `gen_candidates_scipy` with the following warning(s):
[NumericalWarning('A not p.d., added jitter of 1.0e-08 to the diagonal'), OptimizationWarning('Optimization failed within `scipy.optimize.minimize` with status 2 and message ABNORMAL_TERMINATION_IN_LNSRCH.'), NumericalWarning('A not p.d., added jitter of 1.0e-08 to the diagonal')]
Trying again with a new set of initial conditions.
  return _optimize_acqf_batch(opt_inputs=opt_inputs)
/home/kylemcmaster/.local/lib/python3.10/site-packages/botorch/optim/optimize.py:568: RuntimeWarning: Optimization failed on the second try, after generating a new set of initial conditions.
  return _optimize_acqf_batch(opt_inputs=opt_inputs)
[INFO 10-25 14:40:15] ax.service.ax_client: Generated new trial 170 with parameters {'path_penalty': 8.569364, 'loss_penalty': 10.0, 'score_factor': 3.368086} using model GPEI.
[INFO 10-25 14

43704.0 0.9


[INFO 10-25 14:41:01] ax.service.ax_client: Generated new trial 171 with parameters {'path_penalty': 8.225801, 'loss_penalty': 6.057868, 'score_factor': 2.202495} using model GPEI.
[INFO 10-25 14:41:41] ax.service.ax_client: Completed trial 171 with data: {'objective': (0.84, None)}.


38925.92 0.84


/home/kylemcmaster/.local/lib/python3.10/site-packages/botorch/optim/optimize.py:568: RuntimeWarning: Optimization failed in `gen_candidates_scipy` with the following warning(s):
[OptimizationWarning('Optimization failed within `scipy.optimize.minimize` with status 2 and message ABNORMAL_TERMINATION_IN_LNSRCH.'), OptimizationWarning('Optimization failed within `scipy.optimize.minimize` with status 2 and message ABNORMAL_TERMINATION_IN_LNSRCH.'), OptimizationWarning('Optimization failed within `scipy.optimize.minimize` with status 2 and message ABNORMAL_TERMINATION_IN_LNSRCH.')]
Trying again with a new set of initial conditions.
  return _optimize_acqf_batch(opt_inputs=opt_inputs)
/home/kylemcmaster/.local/lib/python3.10/site-packages/botorch/optim/optimize.py:568: RuntimeWarning: Optimization failed on the second try, after generating a new set of initial conditions.
  return _optimize_acqf_batch(opt_inputs=opt_inputs)
[INFO 10-25 14:41:46] ax.service.ax_client: Generated new trial 172

...

/home/kylemcmaster/.local/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(
/home/kylemcmaster/.local/lib/python3.10/site-packages/botorch/optim/optimize.py:568: RuntimeWarning: Optimization failed in `gen_candidates_scipy` with the following warning(s):
[NumericalWarning('A not p.d., added jitter of 1.0e-08 to the diagonal'), OptimizationWarning('Optimization failed within `scipy.optimize.minimize` with status 2 and message ABNORMAL_TERMINATION_IN_LNSRCH.'), NumericalWarning('A not p.d., added jitter of 1.0e-08 to the diagonal'), OptimizationWarning('Optimization failed within `scipy.optimize.minimize` with status 2 and message ABNORMAL_TERMINATION_IN_LNSRCH.'), NumericalWarning('A not p.d., added jitter of 1.0e-08 to the diagonal'), NumericalWarning('A not p.d., added jitter of 1.0e-08 to the diagonal')]
Trying again with a new set of initial conditions.
  return _optimize_acqf_bat

44893.84 0.92


/home/kylemcmaster/.local/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(
/home/kylemcmaster/.local/lib/python3.10/site-packages/botorch/optim/optimize.py:568: RuntimeWarning: Optimization failed in `gen_candidates_scipy` with the following warning(s):
[OptimizationWarning('Optimization failed within `scipy.optimize.minimize` with status 2 and message ABNORMAL_TERMINATION_IN_LNSRCH.')]
Trying again with a new set of initial conditions.
  return _optimize_acqf_batch(opt_inputs=opt_inputs)
/home/kylemcmaster/.local/lib/python3.10/site-packages/botorch/optim/optimize.py:568: RuntimeWarning: Optimization failed on the second try, after generating a new set of initial conditions.
  return _optimize_acqf_batch(opt_inputs=opt_inputs)
[INFO 10-25 15:04:52] ax.service.ax_client: Generated new trial 199 with parameters {'path_penalty': 8.788826, 'loss_penalty': 0.0, 'score_factor': 2.667405}

49692.4 0.88
The best parameters are: {'path_penalty': 8.235771190206512, 'loss_penalty': 0.0, 'score_factor': 3.368975309680944}
with score ({'objective': 0.8783361040228629}, {'objective': {'objective': 6.678842355076609e-05}})


/home/kylemcmaster/.local/lib/python3.10/site-packages/ax/utils/stats/model_fit_stats.py:199: RuntimeWarning: divide by zero encountered in divide
  return float(np.mean(1.96 * 2 * se_pred / np.abs(y_obs)))
/home/kylemcmaster/.local/lib/python3.10/site-packages/ax/utils/stats/model_fit_stats.py:229: RuntimeWarning: divide by zero encountered in scalar divide
  return float((np.max(y_obs) - min_y_obs) / min_y_obs)


In [14]:
best_parameters, metrics = ax_client.get_best_parameters()
print(f'The best parameters are: {best_parameters}')
print(f'with score {metrics}')

data =  ax_client.experiment.fetch_data()
print(data.df)
print(data.df.sort_values(by='mean', ascending=False).head(10))

trial_params = ax_client.experiment.trials[132]
print(trial_params)

The best parameters are: {'path_penalty': 8.505974119306348, 'loss_penalty': 0.0, 'score_factor': 3.4897451874552705}
with score ({'objective': 0.8776559915826968}, {'objective': {'objective': 8.207984467722268e-05}})
    arm_name metric_name  mean  sem  trial_index
0        0_0   objective  0.84  NaN            0
1        1_0   objective  0.84  NaN            1
2        2_0   objective  0.08  NaN            2
3        3_0   objective  0.84  NaN            3
4        4_0   objective  0.84  NaN            4
..       ...         ...   ...  ...          ...
165    165_0   objective  0.86  NaN          165
166    166_0   objective  0.94  NaN          166
167    167_0   objective  0.90  NaN          167
168    168_0   objective  0.84  NaN          168
169    169_0   objective  0.86  NaN          169

[170 rows x 5 columns]
    arm_name metric_name  mean  sem  trial_index
166    166_0   objective  0.94  NaN          166
99      99_0   objective  0.94  NaN           99
72      72_0   objectiv

/home/kylemcmaster/.local/lib/python3.10/site-packages/ax/utils/stats/model_fit_stats.py:199: RuntimeWarning: divide by zero encountered in divide
  return float(np.mean(1.96 * 2 * se_pred / np.abs(y_obs)))
/home/kylemcmaster/.local/lib/python3.10/site-packages/ax/utils/stats/model_fit_stats.py:229: RuntimeWarning: divide by zero encountered in scalar divide
  return float((np.max(y_obs) - min_y_obs) / min_y_obs)
